In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.7 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import torch
import json
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric

torch.backends.cuda.max_split_size_mb = 1024
# Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

In [ ]:
dataset = load_dataset("derek-thomas/ScienceQA")
train_data, val_data = dataset["train"], dataset["validation"]

In [ ]:
%mkdir scienceqa_data

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

vit_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)



max_length = 64
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(example):
  images = []
  # for image_path in image_paths:
  #   i_image = Image.open(image_path)
  #   if i_image.mode != "RGB":
  #     i_image = i_image.convert(mode="RGB")
  image = example["image"]
  if image:
    images.append(image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = vit_model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    example["captions"] = preds[0]
  else:
    example["captions"] = ""
  return example

In [ ]:
#Mapping the predict_step function to generate image captions for training set
train_data_captions = train_data.map(predict_step)

In [ ]:
sample= val_data.to_dict()

In [ ]:
sample1 = val_data.train_test_split(test_size=0.05)["test"]

In [ ]:
len(sample1)

In [ ]:
#Mapping the predict_step function to generate image captions for training set
val_data_captions = sample1.map(predict_step)

In [ ]:
val_data_captions[4]

In [ ]:
type(train_data_captions)

In [ ]:
train_data_captions_dict = train_data_captions.to_dict()

In [ ]:
train_data_captions_dict.keys()

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

# with open('./scienceqa_data/train_captionsdata.p', 'w') as fp:
#     pickle.dump(train_data_captions_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./scienceqa_data/train_captionsdata.pickle', 'wb') as handle:
    pickle.dump(train_data_captions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./scienceqa_data/train_captionsdata.pickle', 'rb') as handle:
    train_data_captions_dict = pickle.load(handle)

In [ ]:
type(train_data_captions_dict)

In [ ]:
train_data_captions.keys()

In [ ]:
train_data_captions_dict = train_data_captions.to_dict()

In [ ]:
val_data_captions_dict = val_data_captions.to_dict()

In [ ]:
with open('./scienceqa_data/valid_captionsdata.pickle', 'wb') as handle:
    pickle.dump(val_data_captions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./scienceqa_data/valid_captionsdata.pickle', 'rb') as handle:
    val_data_captions_dict = pickle.load(handle)

In [ ]:
val_data_captions_dict

In [ ]:
# import json
# with open("./scienceqa_data/valid_captionsdata.json", 'w') as file:
#     json.dump(val_data_captions_dict, file)

In [ ]:
import datasets

In [ ]:
train_data_captions = datasets.Dataset.from_pandas(pd.DataFrame(data=train_data_captions_dict))

In [ ]:
train_data_captions[0]

In [ ]:
val_data_captions = datasets.Dataset.from_pandas(pd.DataFrame(data=val_data_captions_dict))

In [ ]:
val_data_captions[3]

In [ ]:
# Define the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device) 
#Change this to the new model from checkpoints.

In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
# process the examples in input and target text format and the eos token at the end 

#For answer ONLY (not solution and explanation)
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  choices: %s  context: %s </s>' % (example['captions'].lower()+". "+example['question'].lower(), example['choices'], example['skill'].lower()+example['hint'].lower())
    example['target_text'] = 'answer: %s </s>' % (example['choices'][example['answer']])
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512, truncation=True)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=128, truncation=True)
    #max_length = 128 whej just answer, 256 when answer & explanation, 512 when all three
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
# process the examples in input and target text format and the eos token at the end
# Define the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("./models_saved/t5small_answer_model2_512_128_10_8_8").to(device) 
#Change this to the new model from checkpoints.

#For answer and explanation
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  choices: %s  context: %s </s>' % (example['captions'].lower()+". "+example['question'].lower(), example['choices'], example['skill'].lower()+example['hint'].lower())
    example['target_text'] = 'answer: %s  explanation: %s </s>' % (example['choices'][example['answer']], example["solution"])
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512, truncation=True)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=256, truncation=True)
    #max_length = 128 whej just answer, 256 when answer & explanation, 512 when all three
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
# process the examples in input and target text format and the eos token at the end
# Define the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("./models_saved/t5small_answer_model2_512_128_10_8_8").to(device) 
#Change this to the new model from checkpoints.

#For answer with generated explanation in input
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  choices: %s  context: %s </s>' % (example['captions'].lower()+". "+example['question'].lower(), example['choices'], example['generated_exp']+example['hint'].lower())
    example['target_text'] = 'answer: %s </s>' % (example['choices'][example['answer']])
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512, truncation=True)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=128, truncation=True)
    #max_length = 128 whej just answer, 256 when answer & explanation, 512 when all three
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
#For generated examples as input:

# map add_eos_to_examples function to the dataset example wise 
train_dataset = train_data_captions_gen_exp.map(add_eos_to_examples, load_from_cache_file=False)
# map convert_to_features batch wise
train_dataset = train_dataset.map(convert_to_features, batched=True)

valid_dataset = val_data_captions_gen_exp.map(add_eos_to_examples, load_from_cache_file=False)
valid_dataset = valid_dataset.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [ ]:
# map add_eos_to_examples function to the dataset example wise 
train_dataset = train_data_captions.map(add_eos_to_examples, load_from_cache_file=False)
# map convert_to_features batch wise
train_dataset = train_dataset.map(convert_to_features, batched=True)

valid_dataset = val_data_captions.map(add_eos_to_examples, load_from_cache_file=False)
valid_dataset = valid_dataset.map(convert_to_features, batched=True, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [ ]:
len(train_dataset), len(valid_dataset)

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
valid_dataloader = DataLoader(valid_dataset, batch_size=8)

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

In [ ]:
tokenizer.decode(batch['input_ids'][7])

In [ ]:
tokenizer.decode(batch['target_ids'][7])

In [ ]:
from transformers import DataCollator, AdamW, get_linear_schedule_with_warmup
import numpy as np

In [ ]:
def data_collator(batch):
    input_ids = torch.stack([example['input_ids'] for example in batch])
    attention_mask = torch.stack([example['attention_mask'] for example in batch])

    target_ids = torch.stack([example['target_ids'] for example in batch])
    target_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'target_ids': target_ids,
        'target_attention_mask': target_attention_mask
    }

In [ ]:
from tqdm import tqdm
optimizer = AdamW(model.parameters(), lr=1e-5)
num_training_steps = 10000
num_warmup_steps = 500
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Define the training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
train_loss = []
val_loss = []
for epoch in range(10):
    print("Training: ")
    model.train(True)
    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
    total_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=target_ids,
            decoder_attention_mask=target_attention_mask,
            use_cache=False
        )
        
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    train_epoch_loss = total_loss / len(train_dataloader) 
    train_loss.append(train_epoch_loss)
    print(f"Training Loss at epoch {epoch}: {train_epoch_loss}")

    print("Validation:")
    valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_ids = batch['target_ids'].to(device)
            target_attention_mask = batch['target_attention_mask'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=target_ids,
                decoder_attention_mask=target_attention_mask,
                use_cache=False
            )

            loss = outputs.loss
            total_loss += loss.item()
    val_epoch_loss = total_loss / len(valid_dataloader) 
    val_loss.append(val_epoch_loss)
    print(f"Validation Loss at epoch {epoch}: {val_epoch_loss}")
    # val_loss.append(total_loss / len(valid_dataloader))
    # print(val_loss)


In [ ]:
import matplotlib.pyplot as plt
# Plot the training and validation losses
plt.plot(train_loss[0:6], label='Train Loss')
plt.plot(val_loss[0:6], label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Training and Validation Loss for Answer with Generated Explanation as Input (5 Epochs)")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot the training and validation losses
plt.plot(train_loss[1:], label='Train Loss')
plt.plot(val_loss[1:], label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Training and Validation Loss for Answer with Generated Explanation as Input")
plt.legend()
plt.show()

In [ ]:
with open('./scienceqa_data/train_loss_model2', 'wb') as handle:
    pickle.dump(train_loss, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./scienceqa_data/val_loss_model2', 'wb') as handle:
    pickle.dump(val_loss, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.parameters()

In [ ]:
# %mkdir models_saved

Model 6 Plots

In [ ]:
import matplotlib.pyplot as plt
# Plot the training and validation losses
plt.plot(train_loss[0:6], label='Train Loss')
plt.plot(val_loss[0:6], label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Training and Validation Loss for Pre-trained T5 Small for Answer and Explanation Generation (5 Epochs)")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot the training and validation losses
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Training and Validation Loss for Pre-trained T5 Small for Answer and Explanation Generation")
plt.legend()
plt.show()

In [ ]:
with open('./scienceqa_data/train_loss_model6', 'wb') as handle:
    pickle.dump(train_loss, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./scienceqa_data/val_loss_model6', 'wb') as handle:
    pickle.dump(val_loss, handle, protocol=pickle.HIGHEST_PROTOCOL)

Defining the model parameters for model checkpoint saving

In [ ]:
#Parameters for answer only model
input_max_len = 512
target_max_len = 128
num_epochs = 10
# learning_rate = 1e-5
train_batch_size = 8
val_batch_size = 8
# no_of_training_steps = 10000
# no_of_warmup_steps = 500
model_name = f"t5small_answer_model14_{input_max_len}_{target_max_len}_{num_epochs}_{train_batch_size}_{val_batch_size}"

In [ ]:
model_name

In [ ]:

#Answer only model
model.save_pretrained(f"./models_saved/{model_name}", from_pt=True)

In [ ]:
!pip install rouge_score

In [ ]:
# Evaluate the model on the validation set
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer
import string

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
smoothie = SmoothingFunction().method4

model.eval()
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=data_collator)

model_answers_train = []
target_answers_train = []

with torch.no_grad():
    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)
        
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # decoder_attention_mask=target_attention_mask,
            max_length=512,
            num_beams=10,
            early_stopping=True
        )
        
        # Convert the generated output to text
        output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_text = tokenizer.batch_decode(target_ids, skip_special_tokens=True)
        # print(output_text)
        # print("--")
        # print(target_text)
        model_answers_train.append(output_text)
        target_answers_train.append(target_text)

In [ ]:
model_answers_train

In [ ]:
target_answers_train

In [ ]:
# %mkdir results

In [ ]:
with open('./results/model2_answers_train', 'wb') as handle:
    pickle.dump(model_answers_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./results/model2_target_answers_train', 'wb') as handle:
    pickle.dump(target_answers_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./results/model6_answers_train', 'wb') as handle:
    pickle.dump(model_answers_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./results/model6_target_answers_train', 'wb') as handle:
    pickle.dump(target_answers_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./results/model14_answers_train', 'wb') as handle:
    pickle.dump(model_answers_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./results/model14_target_answers_train', 'wb') as handle:
    pickle.dump(target_answers_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

Metrics on Training Set

In [ ]:
model_answers_train_overall,target_answers_train_overall=sum(model_answers_train,[]),sum(target_answers_train,[])

In [ ]:
accuracy_boolean_train=[a.lower().split("explanation:")[0].strip()==b.lower().split("explanation:")[0].strip() for a,b in zip((model_answers_train_overall),(target_answers_train_overall))]

In [ ]:
acc_train=(sum(accuracy_boolean_train)/len(accuracy_boolean_train))*100
print(f"The training accuracy of the model is {acc_train:.2f} %")

In [ ]:
exp_bleu_score_train = []
exp_precision_rouge1_score_train = []
exp_recall_rouge1_score_train = []
exp_precision_rouge2_score_train = []
exp_recall_rouge2_score_train = []
exp_precision_rougeL_score_train = []
exp_recall_rougeL_score_train = []

In [ ]:
def compute_metrics(model_output,target_output):
    out_exp,tar_exp=model_output.split("explanation:")[1].strip(),target_output.split("explanation:")[1].strip()
    exp_bleu_score_train.append(sentence_bleu(tar_exp.split(), out_exp.split(), smoothing_function=smoothie))
    exp_recall_rouge1_score_train.append(scorer.score(tar_exp, out_exp)['rouge1'][1])
    exp_precision_rouge1_score_train.append(scorer.score(tar_exp, out_exp)['rouge1'][0])
    exp_recall_rouge2_score_train.append(scorer.score(tar_exp, out_exp)['rouge2'][1])
    exp_precision_rouge2_score_train.append(scorer.score(tar_exp, out_exp)['rouge2'][0])
    exp_recall_rougeL_score_train.append(scorer.score(tar_exp, out_exp)['rougeL'][1])
    exp_precision_rougeL_score_train.append(scorer.score(tar_exp, out_exp)['rougeL'][0])
    return

In [ ]:
for a,b in zip(model_answers_train_overall,target_answers_train_overall):  
    try:
        compute_metrics(a,b)
    except:
        pass

In [ ]:
print("Training Metrics:")
print("--")
print("For Explanation: ")
print(f"Bleu Score: {np.array(exp_bleu_score_train).mean()}")
print(f"Rouge-1 Precision Score: {np.array(exp_precision_rouge1_score_train).mean()}")
print(f"Rouge-1 Recall Score: {np.array(exp_recall_rouge1_score_train).mean()}")
print(f"Rouge-2 Precision Score: {np.array(exp_precision_rouge2_score_train).mean()}")
print(f"Rouge-2 Recall Score: {np.array(exp_recall_rouge2_score_train).mean()}")
print(f"Rouge-L Precision Score: {np.array(exp_precision_rougeL_score_train).mean()}")
print(f"Rouge-L Recall Score: {np.array(exp_recall_rougeL_score_train).mean()}")


In [ ]:
train_data_captions_gen_exp_dict = train_data_captions_dict

In [ ]:
len(train_data_captions_gen_exp_dict["question"])

In [ ]:
gen_exp_train = [i.lower().split("explanation:")[1].strip() if "explanation:" in i.lower() else '' for i in model_answers_train_overall ]
train_data_captions_gen_exp_dict["generated_exp"] = gen_exp_train
train_data_captions_gen_exp = datasets.Dataset.from_pandas(pd.DataFrame(data=train_data_captions_gen_exp_dict))

In [ ]:
val_data_captions_gen_exp_dict = val_data_captions_dict
gen_exp_val = [i.lower().split("explanation:")[1].strip() if "explanation:" in i.lower() else '' for i in model_answers_overall ]
val_data_captions_gen_exp_dict["generated_exp"] = gen_exp_val
val_data_captions_gen_exp = datasets.Dataset.from_pandas(pd.DataFrame(data=val_data_captions_gen_exp_dict))

Model generation for validation dataset

In [ ]:
# Evaluate the model on the validation set
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer
import string

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
smoothie = SmoothingFunction().method4

model.eval()
valid_dataloader = DataLoader(valid_dataset, batch_size=8, collate_fn=data_collator)

model_answers = []
target_answers = []

with torch.no_grad():
    for batch in tqdm(valid_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)
        
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # decoder_attention_mask=target_attention_mask,
            max_length=512,
            num_beams=10,
            early_stopping=True
        )
        
        # Convert the generated output to text
        output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_text = tokenizer.batch_decode(target_ids, skip_special_tokens=True)
        model_answers.append(output_text)
        target_answers.append(target_text)

In [ ]:
with open('./results/model_2_model_answers', 'wb') as handle:
    pickle.dump(model_answers, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./results/model_2_target_answers', 'wb') as handle:
    pickle.dump(target_answers, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./results/model6_valid_model_answers', 'wb') as handle:
    pickle.dump(model_answers, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./results/model6_valid_target_answers', 'wb') as handle:
    pickle.dump(target_answers, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('./results/model14_valid_model_answers', 'wb') as handle:
    pickle.dump(model_answers, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./results/model14_valid_target_answers', 'wb') as handle:
    pickle.dump(target_answers, handle, protocol=pickle.HIGHEST_PROTOCOL)

Metrics on Validation Set

In [ ]:
model_answers_overall,target_answers_overall=sum(model_answers,[]),sum(target_answers,[])

accuracy_boolean_valid = [a.lower().split("explanation:")[0].strip()==b.lower().split("explanation:")[0].strip() for a,b in zip((model_answers_overall),(target_answers_overall))]

In [ ]:
acc_train=(sum(accuracy_boolean_valid)/len(accuracy_boolean_valid))*100
print(f"The validation accuracy of the model is {acc_train:.2f} %")

In [ ]:
exp_bleu_score_valid = []
exp_precision_rouge1_score_valid = []
exp_recall_rouge1_score_valid = []
exp_precision_rouge2_score_valid = []
exp_recall_rouge2_score_valid = []
exp_precision_rougeL_score_valid = []
exp_recall_rougeL_score_valid = []

In [ ]:
def compute_metrics(model_output,target_output):
    out_exp,tar_exp=model_output.split("explanation:")[1].strip(),target_output.split("explanation:")[1].strip()
    exp_bleu_score_valid.append(sentence_bleu(tar_exp.split(), out_exp.split(), smoothing_function=smoothie))
    exp_recall_rouge1_score_valid.append(scorer.score(tar_exp, out_exp)['rouge1'][1])
    exp_precision_rouge1_score_valid.append(scorer.score(tar_exp, out_exp)['rouge1'][0])
    exp_recall_rouge2_score_valid.append(scorer.score(tar_exp, out_exp)['rouge2'][1])
    exp_precision_rouge2_score_valid.append(scorer.score(tar_exp, out_exp)['rouge2'][0])
    exp_recall_rougeL_score_valid.append(scorer.score(tar_exp, out_exp)['rougeL'][1])
    exp_precision_rougeL_score_valid.append(scorer.score(tar_exp, out_exp)['rougeL'][0])
    return

In [ ]:
for a,b in zip(model_answers_overall,target_answers_overall):  
    try:
        compute_metrics(a,b)
    except:
        pass

In [ ]:
print("Validation Metrics:")
print("--")
print("For Explanation: ")
print(f"Bleu Score: {np.array(exp_bleu_score_valid).mean()}")
print(f"Rouge-1 Precision Score: {np.array(exp_precision_rouge1_score_valid).mean()}")
print(f"Rouge-1 Recall Score: {np.array(exp_recall_rouge1_score_valid).mean()}")
print(f"Rouge-2 Precision Score: {np.array(exp_precision_rouge2_score_valid).mean()}")
print(f"Rouge-2 Recall Score: {np.array(exp_recall_rouge2_score_valid).mean()}")
print(f"Rouge-L Precision Score: {np.array(exp_precision_rougeL_score_valid).mean()}")
print(f"Rouge-L Recall Score: {np.array(exp_recall_rougeL_score_valid).mean()}")
